In [ ]:
import os, glob 
sys.path.append('../')

import SimpleITK as sitk
import numpy as np

In [ ]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patients = os.listdir(src)
if '.DS_Store' in patients:
    patients.remove('.DS_Store')
print(f"Number of patients: {len(patients)}")

In [ ]:
wanted = [ '_predicted', '_resized']
unwanted = ['_3D']

for patient in patients:

    print(f'{patient}')
    
    try:
        folders = glob.glob(src + '/' + patient + '/T2*')
        folders = [x for x in folders if any(y in x for y in wanted) and all(h not in x for h in unwanted)]
        assert len(folders) != 0

    except:
        folders = glob.glob(src + '/' + patient + '/t2*')
        folders = [x for x in folders if any(y in x for y in wanted) and all(h not in x for h in unwanted)]

    for path in folders:
        folder_path = os.path.split(path)[0]
        folder_name = os.path.split(path)[1]
        print(f' folder: {folder_name}')

        images = glob.glob(path + '/*.png')
        
        output_folder = '_'.join((path, '3D'))

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        vect = np.zeros(shape=(len(images), 256, 256))
        for k, i in enumerate(images):
            im = sitk.ReadImage(i)
            im_arr = sitk.GetArrayFromImage(im)
            vect[k, ...] = im_arr

        stack = sitk.GetImageFromArray(vect)

        output_name = 'segmented.nrrd' if wanted[0] in folder_name else 'greyscale.nrrd'

        output = os.path.join(output_folder, output_name)

        if os.path.isfile(output):
            os.remove(output)

        sitk.WriteImage(stack, output)